In [19]:
import tensorflow as tf
from tensorflow import keras

import glob
import cv2
import numpy as np

from sklearn.model_selection import train_test_split

In [11]:
W = H = 75

def preprocessing(img):
    resized = cv2.resize(img,(W,H))
    return resized

In [12]:
classes = []
n_classes = 43
for i in range(n_classes):
    classes.append("%05d"%(i))

In [13]:
data_folder = "/home/namntse05438/datasets/GTSRB/GTSRB_Final_Training_Images/GTSRB/Final_Training/Images/"

# load data set
data = []
target = []

for i, cls in enumerate(classes):
    path = data_folder+cls+"/*.ppm"
    image_paths = glob.glob(path)
    for img_path in image_paths:
        img = cv2.imread(img_path)
        processed_img = preprocessing(img)
        data.append(processed_img)
        target.append(i)

data = np.array(data)
target = np.array(target)

In [14]:
data.shape

(39209, 75, 75, 3)

In [59]:
# define pretrained inception net v3
inception_v2 = keras.applications.inception_resnet_v2.InceptionResNetV2(weights="imagenet", 
                                                           include_top=False, 
                                                           input_shape=(W,H,3))


In [60]:
# compute feature on pretrained network
feature_map = inception_v2.predict(keras.applications.inception_resnet_v2.preprocess_input(data))

In [51]:
# convert label to onehot
oh_target = keras.utils.to_categorical(y=target, num_classes=n_classes)

In [52]:
# build model on top of inception
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dense(units=n_classes, activation='softmax')
])

model.compile(optimizer=tf.train.AdamOptimizer(),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [53]:
# split training and validation data
X_train, X_val, y_train, y_val = train_test_split(feature_map, oh_target, test_size=0.3, random_state=13)

In [61]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

Train on 27446 samples, validate on 11763 samples
Epoch 1/10
27446/27446 [==============================] - 6s 212us/step - loss: 0.1132 - acc: 0.9670 - val_loss: 0.4450 - val_acc: 0.8936
Epoch 2/10
27446/27446 [==============================] - 4s 143us/step - loss: 0.1118 - acc: 0.9678 - val_loss: 0.4845 - val_acc: 0.8871
Epoch 3/10
27446/27446 [==============================] - 4s 145us/step - loss: 0.0993 - acc: 0.9724 - val_loss: 0.4563 - val_acc: 0.8994
Epoch 4/10
27446/27446 [==============================] - 4s 145us/step - loss: 0.0975 - acc: 0.9711 - val_loss: 0.4587 - val_acc: 0.9072
Epoch 5/10
27446/27446 [==============================] - 4s 145us/step - loss: 0.0810 - acc: 0.9765 - val_loss: 0.4520 - val_acc: 0.9046
Epoch 6/10
27446/27446 [==============================] - 4s 145us/step - loss: 0.0981 - acc: 0.9738 - val_loss: 0.4821 - val_acc: 0.8997
Epoch 7/10
27446/27446 [==============================] - 4s 151us/step - loss: 0.1003 - acc: 0.9715 - val_loss: 0.5045 - 

In [79]:
# train everything from scratch
inception_v3 = keras.applications.inception_v3.InceptionV3(weights=None, 
                                                           input_shape=(W,H,3), 
                                                           classes=n_classes)

In [80]:
inception_v3.compile(optimizer=tf.train.AdamOptimizer(),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

In [81]:
inception_v3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_876 (Conv2D)             (None, 37, 37, 32)   864         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_876 (BatchN (None, 37, 37, 32)   96          conv2d_876[0][0]                 
__________________________________________________________________________________________________
activation_876 (Activation)     (None, 37, 37, 32)   0           batch_normalization_876[0][0]    
__________________________________________________________________________________________________
conv2d_877

In [83]:
inception_v3.fit(data, oh_target, validation_split=0.3)

Train on 27446 samples, validate on 11763 samples
Epoch 1/1
27446/27446 [==============================] - 120s 4ms/step - loss: 1.7335 - acc: 0.5219 - val_loss: 14.3911 - val_acc: 0.0122


In [ ]:
# =============>  So you know that training from scratch is terribly awful